## Лабораторная работа №2

### Задание 1

Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.


### Шаг 1. Загрузка датасета

In [1]:
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd


df = pd.read_csv("./base_v3.csv")

y = df["is_fraud"]
X = df.drop("is_fraud", axis=1)

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### Подбор гиперпараметров

C (регуляризация): Определяет силу регуляризации в модели SVM (Support Vector Machine / Машина опорных векторов); более низкое значение C увеличивает штраф за неправильную классификацию, что может привести к созданию простой модели с большей обобщающей способностью. gamma (ядерный коэффициент): Определяет влияние одного тренировочного примера на другие; меньшие значения gamma означают более широкие гауссовы функции и приводят к более простым/прямым границам решения. kernel (ядро): Определяет тип использованного ядра (линейное или RBF); линейное ядро работает линейно, тогда как RBF (радиально-базисная функция) способно моделировать сложные границы решений.

In [2]:
param_grid = {'C': [0.01, 0.1, 1], 
              'kernel': ['linear', 'rbf', 'sigmoid'],
                'gamma': [0.1, 1, 10]}

grid = GridSearchCV(svm.SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_params_

{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}

### Шаг 3. Отчет об убучении модели

In [3]:
best_params = {'kernel': grid.best_params_['kernel'],
               'C': grid.best_params_['C'],
               'gamma': grid.best_params_['gamma']}

model = svm.SVC(**best_params) 
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

report = classification_report(y_test, y_pred, zero_division=1)
print(report)

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(model, X_train, y_train, cv=stratified_kfold)

print(f'Средняя точность перекрестной проверки: {cross_val_scores.mean()}')

Точность модели: 0.9383333333333334
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      2038
           1       0.88      0.68      0.77       362

    accuracy                           0.94      2400
   macro avg       0.91      0.83      0.87      2400
weighted avg       0.94      0.94      0.94      2400

Средняя точность перекрестной проверки: 0.9383333333333332
